In [1]:
# Useful starting lines
%matplotlib inline

import random
from datetime import datetime

import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

# Support Vector Machines
## Classification Using SVM
Load dataset. We will re-use the CERN dataset from project 1, available from https://inclass.kaggle.com/c/epfml-project-1/data

In [2]:
from helpers import load_csv_data

DATA_TRAIN_PATH = r'D:\Documents\etudes\epfl\MA1\cours\MachineLearning\Project1\data\train.csv'

y, X, ids = load_csv_data(DATA_TRAIN_PATH, sub_sample=True)
print('labels y:', y.shape, ', data X:', X.shape)

labels y: (5000,) , data X: (5000, 30)


## Prepare cost and prediction functions

In [3]:
def hinge(Z):
    
    if type(Z) in {int, np.float64}:
        loss = max(0, 1-Z)
        
    if type(Z)==list:
        loss = []
        for z in Z:
            loss.append(max(0, 1-z))
            
    if type(Z)==np.ndarray:
        loss = []
        for z in list(Z):
            loss.append(max(0, 1-z))
        loss = np.array(loss).reshape(Z.shape)
        
    return loss

In [4]:
def calculate_primal_objective(y, X, w, lambda_):
    """compute the full cost (the primal objective), that is loss plus regularizer.
    X: the full dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    """
    
    primal_obj = sum(hinge(np.multiply(y,X.dot(w)))) + (lambda_/2)*w.dot(w)
    
    return primal_obj

In [5]:
from helpers import predict_labels

def calculate_accuracy(y, X, w):
    """compute the training accuracy on the training set (can be called for test set as well).
    X: the full dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    """

    # predict labels
    y_pred = predict_labels(w, X)
    
    # compute accuracy
    acc = len(np.where(np.array(y_pred) - np.array(y) == 0)[0])
    acc = acc / len(y)
    
    return acc

## Stochastic Gradient Descent for SVM

Compute the (stochastic) subgradient for the n-th summand of the SVM optimization objective

In [6]:
def calculate_stochastic_gradient(y, X, w, lambda_, n, num_examples):
    """compute the stochastic gradient of loss plus regularizer.
    X: the dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    n: the index of the (one) datapoint we have sampled
    num_examples: N
    """
    # Be careful about the constant N (size) term!
    # The complete objective for SVM is a sum, not an average as in earlier SGD examples!
    
    # get picked sample
    x_n, y_n = X[n,:], y[n]

    # compute SG: term with the sum
    if y_n*x_n.T.dot(w)>=1:
        sub_grad = 0
    else:
        sub_grad = -y_n*x_n

    # compute SG: term regularization
    SG = sub_grad + lambda_*w

    return SG

Implement stochastic gradient descent: Pick a data point uniformly at random and update w based on the gradient for the n-th summand of the objective

In [7]:
def sgd_for_svm_demo(y, X):
    
    max_iter = int(1e6)
    gamma = 1
    lambda_ = 0.01
    
    num_examples, num_features = X.shape
    w = np.zeros(num_features)
    
    for it in range(max_iter):
        
        # n = sample one data point uniformly at random data from X
        n = random.randint(0,num_examples-1)
        
        # compute the SG
        grad = calculate_stochastic_gradient(y, X, w, lambda_, n, num_examples)
        
        # update w
        w -= gamma/(it+1) * grad
        
        if it % (max_iter/10) == 0:
            cost = calculate_primal_objective(y, X, w, lambda_)
            print("iteration={i}, cost={c}".format(i=it, c=cost))
    
    print("training accuracy = {l}".format(l=calculate_accuracy(y, X, w)))

sgd_for_svm_demo(y, X)

iteration=0, cost=877206628.4637158
iteration=100000, cost=37264936.35535651
iteration=200000, cost=33492068.026689254
iteration=300000, cost=31287530.473561518
iteration=400000, cost=29803757.11257065
iteration=500000, cost=28665819.684582062
iteration=600000, cost=27779467.9826672
iteration=700000, cost=27028162.900114067
iteration=800000, cost=26402414.167239055
iteration=900000, cost=25835766.29123702
training accuracy = 0.6764


## Coordinate Descent (Ascent) for SVM

Compute the closed-form update for the n-th variable alpha, in the dual optimization problem, given alpha and the current corresponding w

In [66]:
def calculate_coordinate_update(y, X, lambda_, alpha, w, n):
    """compute a coordinate update (closed form) for coordinate n.
    X: the dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    n: the coordinate to be updated
    """

    # calculate the update of coordinate at index=n.
    x_n, y_n = X[n], y[n]
    old_alpha_n = np.copy(alpha[n])
    
    # if new coordinate to 0
    alpha[n] = 0
    obj0 = calculate_dual_objective(y, X, w, alpha, lambda_)
    
    # if new coordinate to 1
    alpha[n] = 1
    obj1 = calculate_dual_objective(y, X, w, alpha, lambda_)
    
    # update alpha
    if obj0 > obj1:
        alpha[n] = 0
    else:
        alpha[n] = 1
    
    # update w
    w = (1/lambda_)*X.T.dot(np.diag(y)).dot(alpha)
    
    return w, alpha

In [67]:
def calculate_dual_objective(y, X, w, alpha, lambda_):
    """calculate the objective for the dual problem."""
    
    dual_obj = sum(np.multiply(alpha, 1-np.multiply(y, X.dot(w)))) + (lambda_/2)*w.dot(w)
    
    return dual_obj

In [68]:
def coordinate_descent_for_svm_demo(y, X):
    max_iter = int(1e3)
    lambda_ = 0.01

    num_examples, num_features = X.shape
    w = np.zeros(num_features)
    alpha = np.zeros(num_examples)
    
    for it in range(max_iter):
        
        # n = sample one data point uniformly at random data from x
        n = random.randint(0,num_examples-1)
        
        # update alpha and w w.r.t to the coordinate alpha[n]
        w, alpha = calculate_coordinate_update(y, X, lambda_, alpha, w, n)
            
        if it % (max_iter/10) == 0:
            # primal objective
            primal_value = calculate_primal_objective(y, X, w, lambda_)
            # dual objective
            dual_value = calculate_dual_objective(y, X, w, alpha, lambda_)
            # primal dual gap
            duality_gap = primal_value - dual_value
            print('iteration=%i, primal:%.5f, dual:%.5f, gap:%.5f'%(
                    it, primal_value, dual_value, duality_gap))
    print("training accuracy = {l}".format(l=calculate_accuracy(y, X, w)))

coordinate_descent_for_svm_demo(y, X)

iteration=0, primal:65959191962.51595, dual:-25469915.12555, gap:65984661877.64150
iteration=100, primal:220866421673.72037, dual:-226903051.09580, gap:221093324724.81616
iteration=200, primal:123484837560.59338, dual:-146283277.80920, gap:123631120838.40259


KeyboardInterrupt: 

In [70]:
alpha = np.ones(y.shape)
lambda_ = 0.1
w = np.zeros(X.shape[1])

print('y', y.shape)
print('X', X.shape)
print('w', w.shape)
print('alpha', alpha.shape)

print(calculate_dual_objective(y, X, w, alpha, lambda_))

y (5000,)
X (5000, 30)
w (30,)
alpha (5000,)
-14003707.771
